<a href="https://colab.research.google.com/github/ChloeZhou1997/BreastCancerCNN/blob/main/Breast_Cancer_Classfication_VGG_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pylibjpeg==1.1.1
!pip install pydicom==2.1.1
!pip install torchmetrics
!pip install pytorch_lightning
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import random

In [3]:
normalizer = np.load('/content/drive/MyDrive/Project/normalizer.npy')
mean = normalizer[0]
std = normalizer[1]

#Data Loading

In [4]:
import torch
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
# from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.notebook import tqdm
import numpy as np

In [5]:
def load_file(path):
  return np.load(path).astype(np.float32)

In [6]:
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,std),
                                       ]
)

val_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,std),                                     
])

In [7]:
train_dataset = torchvision.datasets.DatasetFolder("/content/drive/MyDrive/Data/processed/training",
                                                   loader = load_file,
                                                   extensions = "npy",
                                                   transform = train_transforms)

val_dataset = torchvision.datasets.DatasetFolder("/content/drive/MyDrive/Data/processed/validation",
                                                   loader = load_file,
                                                   extensions = "npy",
                                                   transform = train_transforms)

In [8]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

# Model Creation

In [9]:
# torchvision.models.vgg19();

In [10]:
import torchmetrics

class CheastCancer_VGG(pl.LightningModule):

  def __init__(self,init_weights=True):
    super().__init__()

    self.model = torchvision.models.vgg19()
    self.model.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    self.model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=1, bias=True)
    
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 1e-3)

    #loss function
    self.criterion = torch.nn.BCEWithLogitsLoss()

    #eval matrix
    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()

    #list to store loss curve and accuracy curve
    self.traincc, self.valacc = [],[]
    self.trainloss, self.valloss = [],[0]

  def forward(self, data):
    pred = self.model(data)
    return pred

  def training_step(self, batch, batch_idx):
    img, label = batch
    pred = self(img)
    # print(pred)
    # label = label.float()
    loss = self.criterion(pred[:,0],label.float())
    acc = self.train_acc(pred[:,0],label)

    print(f"Training Step : The Loss is {loss} and the accuracy is {acc}")

    self.log("Train Loss", loss)
    self.log("Step Train ACC", self.train_acc(torch.sigmoid(pred[:,0]), label.int()))

    return loss
  
  def training_epoch_end(self, outs):
    self.log("Train ACC", self.train_acc.compute())

  def validation_step(self, batch, batch_idx):
    img, label = batch
    # label = label.float()
    pred = self(img)
    # print(img)
    # print(pred)
    # loss = self.loss_fn(pred,label)
    loss = self.criterion(pred[:,0],label.float())
    acc = self.val_acc(torch.sigmoid(pred[:,0]), label.int())

    print(f"Validation Step : The Loss is {loss} and the accuracy is {acc}")

    self.log("Train Loss", loss)
    self.log("Step Train ACC", acc)


  def validation_epoch_end(self, outs):
    self.log("Val ACC", self.val_acc.compute())

  def configure_optimizers(self):
      return [self.optimizer]

#Model Trainig

In [11]:
def evaluation(model, dataloader):
  predictions, labels = [], []
  with torch.no_grad():
    for batch in dataloader:
      batch = [term.cuda() for term in batch]
      pred = model(batch[0]).squeeze(1)
      pred = torch.sigmoid(pred)
      pred = [1 if p >= 0.5 else 0 for p in pred]
      predictions += pred
      labels += batch[1]
  
  result = 0
  for p, l in zip(predictions, labels):

    if p == l:
      result += 1
    else:
      continue
  accuracy = result / len(labels)
  return accuracy

In [12]:
import tqdm
best_accuracy = float('-inf')
# state_dict = torch.load('file_path')
# model.load_state_dict(state_dict)
step = 0
patinet = 0
stop_train = False
model = CheastCancer_VGG()
model = model.cuda()
for epoch in range(50):
  bar = tqdm.trange(len(train_loader))
  total_loss = 0
  for epoch_step, batch in zip(bar, train_loader):
    model.optimizer.zero_grad()
    model.train()
    batch = [term.cuda() for term in batch]

    pred = model(batch[0])
    loss = model.criterion(pred.squeeze(1), batch[1].float())

    loss.backward()
    total_loss += loss.item()
    model.optimizer.step()
    bar.set_postfix(avg_loss='{}'.format(total_loss / (epoch_step+1)))

    if step % 500 == 0 and step != 0:
      model.eval()
      val_accu = evaluation(model, val_loader)
      print('Val Accuracy: {}'.format(val_accu))

      if val_accu >= best_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Project/checkpoint/VGG/checkpoint_ACC_{}.ckpt'.format(val_accu))
        patient = 0
      else:
        patient += 1
      
      if patient == 5:
        stop_train = True
        break
    step += 1
  if stop_train:
    print('Stop Training !')
    break

 67%|██████▋   | 201/299 [02:27<01:11,  1.37it/s, avg_loss=0.6960051068575075]

Val Accuracy: 0.5174499767333643


 34%|███▍      | 103/299 [01:15<02:22,  1.38it/s, avg_loss=0.6921390707676227]

Val Accuracy: 0.5174499767333643


  2%|▏         | 5/299 [00:04<03:30,  1.40it/s, avg_loss=0.6932323575019836]

...

 82%|████████▏ | 246/299 [02:56<00:38,  1.38it/s, avg_loss=0.6925854967673298]

Val Accuracy: 0.5174499767333643


 49%|████▉     | 148/299 [01:46<01:46,  1.41it/s, avg_loss=0.6927562968042873]

Val Accuracy: 0.5174499767333643


100%|██████████| 299/299 [03:34<00:00,  1.39it/s, avg_loss=0.6924987333674096]


In [13]:
# model2 = CheastCancer_VGG()

In [14]:
# checkpoint_callback2 = ModelCheckpoint(
#     dirpath = "/content/drive/MyDrive/Project/checkpoint/VGG",
#     filename="sample-breastcancer-{epoch:02d}-{Val ACC:.2f}",
#     monitor = "Val ACC",
#     save_top_k = 10,
#     mode ="max")

In [15]:
# gpus = 1
# trainer2 = pl.Trainer(gpus = gpus, logger = TensorBoardLogger(save_dir = "/content/drive/MyDrive/Project/logs/VGG"), log_every_n_steps = 1,
#                      callbacks = checkpoint_callback2, max_epochs = 600)

In [16]:
# trainer2.fit(model2, train_loader, val_loader)